<a href="https://colab.research.google.com/github/life-ofpi/SDATP/blob/main/sda_project_jinseo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns

train= pd.read_csv('/content/drive/MyDrive/dacon/전력예측_sda/train.csv')
building_info = pd.read_csv('/content/drive/MyDrive/dacon/전력예측_sda/building_info.csv')


In [5]:
train

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [6]:
train= train.rename(columns={
    '건물번호': 'building_id',
    '일시': 'date_time',
    '기온(C)': 'temp',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption',
})
train.drop('num_date_time', axis=1, inplace=True)
train.head()


,building_id,date_time,temp,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption
0,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [7]:
building_info = building_info.rename(columns={
    '건물번호': 'building_id',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_cap',
    'ESS저장용량(kWh)': 'ess_cap',
    'PCS용량(kW)': 'pcs_cap',
})
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}
building_info['building_type'] = building_info['building_type'].replace(translation_dict)
building_info.head()


,building_id,building_type,total_area,cooling_area,solar_power_cap,ess_cap,pcs_cap
0,1,Other Buildings,110634.00,39570.00,-,-,-
1,2,Other Buildings,122233.47,99000.00,-,-,-
2,3,Other Buildings,171243.00,113950.00,40,-,-
3,4,Other Buildings,74312.98,34419.62,60,-,-
4,5,Other Buildings,205884.00,150000.00,-,2557,1000


In [8]:
train= pd.merge(train, building_info, on='building_id', how='left')


In [9]:
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')
train['hour'] = train['date_time'].dt.hour
train['day'] = train['date_time'].dt.day
train['month'] = train['date_time'].dt.month
train['year'] = train['date_time'].dt.year
train.head()
#월,주,시간,날로 변경

,building_id,date_time,temp,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,building_type,total_area,cooling_area,solar_power_cap,ess_cap,pcs_cap,hour,day,month,year
0,1,2022-06-01 00:00:00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,Other Buildings,110634.0,39570.0,-,-,-,0,1,6,2022
1,1,2022-06-01 01:00:00,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,Other Buildings,110634.0,39570.0,-,-,-,1,1,6,2022
2,1,2022-06-01 02:00:00,17.7,NaN,1.5,45.0,NaN,NaN,974.88,Other Buildings,110634.0,39570.0,-,-,-,2,1,6,2022
3,1,2022-06-01 03:00:00,16.7,NaN,1.4,48.0,NaN,NaN,953.76,Other Buildings,110634.0,39570.0,-,-,-,3,1,6,2022
4,1,2022-06-01 04:00:00,18.4,NaN,2.8,43.0,NaN,NaN,986.40,Other Buildings,110634.0,39570.0,-,-,-,4,1,6,2022


In [10]:
#결측치
train.isna().sum()
#rainfall, sunshine, solar_radiation가 결측치 다수

building_id               0
date_time                 0
temp                      0
rainfall             160069
windspeed                19
humidity                  9
sunshine              75182
solar_radiation       87913
power_consumption         0
building_type             0
total_area                0
cooling_area              0
solar_power_cap           0
ess_cap                   0
pcs_cap                   0
hour                      0
day                       0
month                     0
year                      0
dtype: int64

In [11]:
#강수량의 결측치는 주로 비가 오지 않은 시간을 의미할 수 있으므로 0으로
train['rainfall'] = train['rainfall'].fillna(0)
train

,building_id,date_time,temp,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,building_type,total_area,cooling_area,solar_power_cap,ess_cap,pcs_cap,hour,day,month,year
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,NaN,NaN,1085.28,Other Buildings,110634.00,39570.00,-,-,-,0,1,6,2022
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,NaN,NaN,1047.36,Other Buildings,110634.00,39570.00,-,-,-,1,1,6,2022
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,NaN,NaN,974.88,Other Buildings,110634.00,39570.00,-,-,-,2,1,6,2022
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,NaN,NaN,953.76,Other Buildings,110634.00,39570.00,-,-,-,3,1,6,2022
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,NaN,NaN,986.40,Other Buildings,110634.00,39570.00,-,-,-,4,1,6,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,NaN,881.04,Hotel and Resort,57497.84,40035.23,-,-,-,19,24,8,2022
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,NaN,798.96,Hotel and Resort,57497.84,40035.23,-,-,-,20,24,8,2022
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,NaN,NaN,825.12,Hotel and Resort,57497.84,40035.23,-,-,-,21,24,8,2022
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,NaN,NaN,640.08,Hotel and Resort,57497.84,40035.23,-,-,-,22,24,8,2022


In [13]:
# 시간대별 평균으로 결측치 채워보자
sunshine_hourly_avg = train.groupby('hour')['sunshine'].mean()
solar_radiation_hourly_avg = train.groupby('hour')['solar_radiation'].mean()

# 결측치에만 평균값 적용
train.loc[train['sunshine'].isnull(), 'sunshine'] = train['hour'].map(sunshine_hourly_avg)
train.loc[train['solar_radiation'].isnull(), 'solar_radiation'] = train['hour'].map(solar_radiation_hourly_avg)


In [14]:
train

,building_id,date_time,temp,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,building_type,total_area,cooling_area,solar_power_cap,ess_cap,pcs_cap,hour,day,month,year
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.000000,1085.28,Other Buildings,110634.00,39570.00,-,-,-,0,1,6,2022
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.000000,1047.36,Other Buildings,110634.00,39570.00,-,-,-,1,1,6,2022
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.000000,974.88,Other Buildings,110634.00,39570.00,-,-,-,2,1,6,2022
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.000000,953.76,Other Buildings,110634.00,39570.00,-,-,-,3,1,6,2022
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.000000,986.40,Other Buildings,110634.00,39570.00,-,-,-,4,1,6,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.300450,881.04,Hotel and Resort,57497.84,40035.23,-,-,-,19,24,8,2022
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.043287,798.96,Hotel and Resort,57497.84,40035.23,-,-,-,20,24,8,2022
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.000000,825.12,Hotel and Resort,57497.84,40035.23,-,-,-,21,24,8,2022
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.000000,640.08,Hotel and Resort,57497.84,40035.23,-,-,-,22,24,8,2022


In [15]:
print(len(train[train['solar_power_cap'] == '-']))
print(len(train[train['ess_cap'] == '-']))
print(len(train[train['pcs_cap'] == '-']))
#없애줘야함

130560
193800
193800


In [16]:
train = train.drop(['solar_power_cap', 'ess_cap', 'pcs_cap'], axis=1)

In [17]:
#building type이 범주형이기 때문에 수치형으로 변환해준다
train= pd.get_dummies(train, columns=['building_type'])


In [20]:
#서로 다른 스케일을 가진 수치형 feature 스케일을 조정
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
columns_to_scale = ['temp', 'rainfall', 'windspeed', 'humidity']
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])


In [21]:
train

,building_id,date_time,temp,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,total_area,...,building_type_Data Center,building_type_Department Store and Outlet,building_type_Discount Mart,building_type_Hospital,building_type_Hotel and Resort,building_type_Knowledge Industry Center,building_type_Other Buildings,building_type_Public,building_type_Research Institute,building_type_University
0,1,2022-06-01 00:00:00,-1.837147,-0.173062,-0.908642,-2.434756,0.0,0.000000,1085.28,110634.00,...,0,0,0,0,0,0,1,0,0,0
1,1,2022-06-01 01:00:00,-1.995898,-0.173062,-0.761378,-2.235818,0.0,0.000000,1047.36,110634.00,...,0,0,0,0,0,0,1,0,0,0
2,1,2022-06-01 02:00:00,-2.075273,-0.173062,-0.466850,-2.235818,0.0,0.000000,974.88,110634.00,...,0,0,0,0,0,0,1,0,0,0
3,1,2022-06-01 03:00:00,-2.339857,-0.173062,-0.540482,-2.036880,0.0,0.000000,953.76,110634.00,...,0,0,0,0,0,0,1,0,0,0
4,1,2022-06-01 04:00:00,-1.890064,-0.173062,0.490366,-2.368443,0.0,0.000000,986.40,110634.00,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,-0.646518,-0.173062,-0.908642,0.483002,0.5,0.300450,881.04,57497.84,...,0,0,0,0,1,0,0,0,0,0
203996,100,2022-08-24 20:00:00,-0.831727,-0.173062,-0.614114,0.483002,0.0,0.043287,798.96,57497.84,...,0,0,0,0,1,0,0,0,0,0
203997,100,2022-08-24 21:00:00,-1.122770,-0.173062,-0.835010,0.880878,0.0,0.000000,825.12,57497.84,...,0,0,0,0,1,0,0,0,0,0
203998,100,2022-08-24 22:00:00,-1.202145,-0.173062,-1.350434,1.013504,0.0,0.000000,640.08,57497.84,...,0,0,0,0,1,0,0,0,0,0
